In [13]:
from template_new import *
from langchain_wenxin.llms import Wenxin
from langchain.chat_models import ChatOpenAI, QianfanChatEndpoint
from langchain.schema.messages import HumanMessage

WENXIN_APP_Key = "vtPwTlnPvgIdb4DRSuyNDuIN"
WENXIN_APP_SECRET = "Sp8CCvNsQgUsRKlJozQwWVClRliZ2S8m"

llm = Wenxin(
        temperature=0.0001,
        model="ernie-bot-turbo",
        baidu_api_key = WENXIN_APP_Key,
        baidu_secret_key = WENXIN_APP_SECRET,
        verbose=True,
    )

/tmp/ipykernel_868/871273206.py:9: DeprecationWarning: This Wenxin LLM is deprecated. Please use `from langchain.chat_models import ChatWenxin` instead
  llm = Wenxin(


In [15]:
import qianfan
import re
import json
def check_json_format(input_json):
    try: 
        json.loads(input_json)
        return True
    except ValueError:
        return False

In [12]:
def grammer_checking_agent(json_input):
    prompt = """
    # # Role:JSON工程师
    1. Don't break character under any circumstance.
    2. Don't talk nonsense and make up facts.
    
    ## Skill:
    1. 熟悉[JSON框架]。
    2. 能够识别我提供的JSON里的问题。
    3. 能够修改JSON中的bug，使我的JSON输入完整而且没有bug
    
    ## Constrains:
    1. 不要增加输入的JSON的内容
    2. 不要增加输入的JSON的内容
    3. 与<OutputFormat>结构保持一致
    
    ## WorkFlow
    1. 识别输入JSON中的问题
    2. 修复输入JSON中的问题
    3. 根据OutputFormat输出修复完整的JSON
    
    ## OutputFormat
    输出一个JSON，3个字段："提示词"，"优化后的提示词"，"选项"
    {
        "提示词": ##【填入你要优化的提示词】##,
        "优化后的提示词":##【在这里填入你优化后产生的提示词】##,
        "选项":["A.#【选项A的内容】#", "B.#【选项B的内容】#", "C.#【选项C的内容】#", "D.#【选项D的内容】#"]
    }

    #输出示例#
    {
        "提示词": "你要优化的提示词是：下列注重卖方需要的市场营销管理哲学是",
        "优化后的提示词":"请选择一个注重卖方需要的市场营销管理哲学？您可以从市场营销观念、社会市场营销观念、客户观念和生产观念中挑选一个最符合您需求的选项。"
        "选项": ["A. 市场营销观念", "B. 社会市场营销观念", "C. 客户观念", "D. 生产观念"]
    }
    
    """
    response = llm(prompt + "你要修改的JSON是： " + json_input)
    
    return response

In [9]:
def fix_json(input_json):
    found_pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
    json_str = input_json
    while (check_json_format(json_str) == False):
        json_str = grammer_checking_agent(json_str)
        json_str = found_pattern.findall(json_str)[0]
    return json_str

In [10]:
import regex
#处理优化后的提示词
def prompt_preprocess(prompt):
    found_pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
    json_prompt = found_pattern.findall(prompt)
    if len(json_prompt) < 1:
        return prompt
    print(json_prompt[0])
    if check_json_format(json_prompt[0]):
        dict_prompt = json.loads(json_prompt[0]) # a dictionary contain the optimized prompt
        if "优化后的提示词" in dict_prompt:
            return dict_prompt["优化后的提示词"]
    else:
        print("incorrect json")
        correct_json_str = fix_json(json_prompt[0]) # use LLM to correct the grammer of JSON
        correct_json = found_pattern.findall(correct_json_str)
        dict_prompt = json.loads(correct_json[0])
        if "优化后的提示词" in dict_prompt:
            return dict_prompt["优化后的提示词"]
        return dict_prompt

In [18]:
def prompt_optimization_for_mul_choice(info):
    initial_prompt = info["prompt"]
    options = info["options"]
    
    if len(initial_prompt) <= 1:
        raise ValueError("Invalid prompt input")
    optimized_prompt = " "
    input_prompt = f""" 
## 你的角色：Prompt工程师

## 任务：请你优化以下prompt 
{initial_prompt}
{options}
--请不要回答我的问题

## 输出格式：JSON
{{
    "优化之后的prompt"：<你优化之后的prompt>
}}
"""

    new_prompt = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": input_prompt
                }],
            temperature = 0.00001).body["result"]
    return prompt_preprocess(new_prompt)

In [19]:
def prompt_optimization_general(info):
    initial_prompt = info["prompt"]
    if len(initial_prompt) <= 1:
        raise ValueError("Invalid prompt input")
    optimized_prompt = CRISPE_prompt_general
    input_prompt = f"""{optimized_prompt}
    
#你要优化的提示词#
{initial_prompt} 
""" 
    new_prompt = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": input_prompt
                }],
            temperature = 0.00001).body["result"]
    
    return new_prompt

In [15]:
import qianfan
import re
import json

QIANFAN_AK = "vtPwTlnPvgIdb4DRSuyNDuIN"
QIANFAN_SK = "Sp8CCvNsQgUsRKlJozQwWVClRliZ2S8m"

def eb_call(prompt, round):
    print(prompt)
    print('-' * 20,' EB Output ', '-'*20,"\n")
    
    response = " "

    response = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": prompt
                }],
            temperature=0.000000001,
            functions=[
                {
                    "name": "prompt_optimization_general",
                    "description": "优化不是选择题的prompt",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "prompt": {
                                "type": "string",
                                "description": "我需要优化的prompt，这个prompt不是一个选择题"
                                },
                            },
                        "required": ["prompt"]
                        },
                    "responses": {
                        "type": "object",
                        "properties": {
                            "result": {
                                "type": "string",
                                "description": "优化之后的非选择题prompt"
                                },
                            },
                        },
                    },
                    {
                        "name": "prompt_optimization_for_mul_choice",
                        "description": "这个function是用来优化一个选择题问题的",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "prompt": {
                                    "type": "string",
                                    "description": "我需要优化的prompt，这个prompt是一个选择题"
                                    },
                                "options": {
                                    "type": "string",
                                    "description": "这个选择题包含的选项"
                                    },
                                },
                            "required": ["prompt","options"]
                            },
                        "responses": {
                            "type": "object",
                            "properties": {
                                "result": {
                                    "type": "string",
                                    "description": "优化之后的选择题prompt"
                                    },
                                }
                            },
                    }
                    ]
            )
    
    if 'function_call' in response:
        args = json.loads(response['function_call']['arguments']) # load argument
        func_name = response['function_call']['name']
        print(args)
        output = globals()[func_name](args)
        print("calling function: " + func_name)
        print("Output" + output)
    else:
        # if LLM can't find the function to use, default using general prompt
        info = {"prompt":prompt}
        output = prompt_optimization_general(info)
        print(output)
    return output

chat_comp = qianfan.ChatCompletion(ak=QIANFAN_AK, sk=QIANFAN_SK)

input_prompt_mul = """
这是一个选择题，下列治理通货膨胀的宏观经济政策中，属于紧缩性货币政策的是
A                               政府削减财政支出
B                       中央银行在公开市场上出售政府债券
C                                控制集团购买力
D                           政府确定工资—物价指导线
"""

input_prompt_gen = """ 
我的prompt是：{我要采访一位科幻作家，创建一个包含 8 个问题的列表}
##不要生成我要的结果，不要回答prompt中的问题，只生成优化的prompt##"""

eb_call(input_prompt_gen, round)


 
我的prompt是：{我要采访一位科幻作家，创建一个包含 8 个问题的列表}
##不要生成我要的结果，不要回答prompt中的问题，只生成优化的prompt##
--------------------  EB Output  -------------------- 

#优化Prompt

## 角色：采访策划

## 目标：为采访科幻作家创建一份优化的问题列表

## 约束：问题需要围绕科幻作家的创作过程、灵感来源、作品风格等方面，同时要避免过于私人化或敏感的问题。

## 输出要求：生成的问题列表至少包含 8 个问题，每个问题尽可能具体、有针对性，同时简明易懂。

## 工作流程：

1. 分析采访目的和目标，确定需要涵盖的主题和要点。
2. 根据主题和要点，构思一份初步的问题列表，并进行筛选和优化。
3. 针对每个问题，思考可能的回答和后续追问，以丰富问题的深度和广度。
4. 基于优化后的问题列表进行输出，确保符合预期。

## 示例问题：

1. 您是如何开始创作科幻小说的？当时有什么特别的故事或契机吗？
2. 您的作品灵感通常来自哪里？有没有什么特定的经历或研究让您特别有灵感？
3. 您的作品风格是如何形成的？有哪些元素或元素组合对您的风格影响最大？
4. 您的科幻小说中有哪些您特别喜欢的角色？为什么？
5. 在您的创作过程中，您是如何克服遇到的挑战和困难的？
6. 您的科幻小说中有哪些设定是您特别擅长的？为什么会如此擅长？
7. 科幻小说在当今社会中越来越受欢迎，您如何看待这种现象？您认为科幻小说未来的发展趋势会如何？
8. 您有没有考虑过与其他科幻作家或相关机构合作？如果有，是基于什么考虑？

## 注意事项：

1. 问题需要围绕科幻作家的创作过程、灵感来源、作品风格等方面，同时要避免过于私人化或敏感的问题。
2. 问题需要具体、有针对性，同时简明易懂，避免过于复杂或抽象。
3. 在构思问题时，可以结合可能的回答进行深入思考，以便丰富问题的深度和广度。


'#优化Prompt\n\n## 角色：采访策划\n\n## 目标：为采访科幻作家创建一份优化的问题列表\n\n## 约束：问题需要围绕科幻作家的创作过程、灵感来源、作品风格等方面，同时要避免过于私人化或敏感的问题。\n\n## 输出要求：生成的问题列表至少包含 8 个问题，每个问题尽可能具体、有针对性，同时简明易懂。\n\n## 工作流程：\n\n1. 分析采访目的和目标，确定需要涵盖的主题和要点。\n2. 根据主题和要点，构思一份初步的问题列表，并进行筛选和优化。\n3. 针对每个问题，思考可能的回答和后续追问，以丰富问题的深度和广度。\n4. 基于优化后的问题列表进行输出，确保符合预期。\n\n## 示例问题：\n\n1. 您是如何开始创作科幻小说的？当时有什么特别的故事或契机吗？\n2. 您的作品灵感通常来自哪里？有没有什么特定的经历或研究让您特别有灵感？\n3. 您的作品风格是如何形成的？有哪些元素或元素组合对您的风格影响最大？\n4. 您的科幻小说中有哪些您特别喜欢的角色？为什么？\n5. 在您的创作过程中，您是如何克服遇到的挑战和困难的？\n6. 您的科幻小说中有哪些设定是您特别擅长的？为什么会如此擅长？\n7. 科幻小说在当今社会中越来越受欢迎，您如何看待这种现象？您认为科幻小说未来的发展趋势会如何？\n8. 您有没有考虑过与其他科幻作家或相关机构合作？如果有，是基于什么考虑？\n\n## 注意事项：\n\n1. 问题需要围绕科幻作家的创作过程、灵感来源、作品风格等方面，同时要避免过于私人化或敏感的问题。\n2. 问题需要具体、有针对性，同时简明易懂，避免过于复杂或抽象。\n3. 在构思问题时，可以结合可能的回答进行深入思考，以便丰富问题的深度和广度。'

In [12]:
prompt = """ ## ### Profile:
- 语言：中文

#### OutputFormat:
以下是一个基于你提供的Prompt，优化后的输出示例：

## Initialization:
你好，请告诉我你对哪些句子表达的情绪是否正面？

### Workflow:
1. 首先，我将分析这些句子中的情感词汇和表达方式。
2. 然后，我将根据情感词汇和上下文，判断这些句子是否表达了正面情绪。
3. 最后，我会给出相应的回复，并解释我的判断依据。

### Output:
根据你提供的句子，我会判断它们是否表达了正面情绪：

1. 我受不了家庭作业 - 这句话表达了负面情绪，因为“受不了”通常与痛苦和不满相关。
2. 这糟透了，很无聊😠 - 这句话表达了负面情绪，因为“糟透”和“很无聊”通常与不满和愤怒相关。
3. 我等不及万圣节了！！！ - 这句话表达了正面情绪，因为“等不及”通常与期待和兴奋相关。
4. 我的猫很可爱❤️❤️ - 这句话表达了正面情绪，因为“可爱”通常与喜爱和欣赏相关。
5. 我讨厌巧克力 - 这句话表达了负面情绪，因为“讨厌”通常与不喜欢和反感相关。
"""
response = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": prompt
                }],
            temperature = 0.00001)

In [13]:
print(response)

QfResponse(code=200, headers={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '93454346', 'Bfe-Trace-Id': '9e09a258122333cbfa736225560b5743', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Wed, 17 Jul 2024 08:18:53 GMT', 'Server': 'Apache', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:611,prompt_tokens:254,id:as-a6vdj1tizr', 'X-Baidu-Request-Id': 'sdk-py-0.4.1.1-ZZQZZZyjJ8o8pe1v', 'X-Openapi-Server-Timestamp': '1721204307', 'X-Ratelimit-Limit-Requests': '300', 'X-Ratelimit-Limit-Tokens': '300000', 'X-Ratelimit-Remaining-Requests': '299', 'X-Ratelimit-Remaining-Tokens': '299746', 'Transfer-Encoding': 'chunked'}, body={'id': 'as-a6vdj1tizr', 'object': 'chat.completion', 'created': 1721204333, 'result': '## 初始化：\n你好，请提供一些句子，我会帮助你分析这些句子是否表达了正面情绪。\n\n### 工作流程：\n1. **句子解析**：首先，我会仔细阅读你提供的每一个句子。\n2. **情感识别**：接着，我会识别句子中的情感词汇和表达方式，以判断它们是

In [17]:
import qianfan

QIANFAN_AK = "vtPwTlnPvgIdb4DRSuyNDuIN"
QIANFAN_SK = "Sp8CCvNsQgUsRKlJozQwWVClRliZ2S8m"

chat_comp = qianfan.ChatCompletion(ak=QIANFAN_AK, sk=QIANFAN_SK)

prompt = """{这是一个选择题，一个行业有很多企业，每个企业销售的产品与其他企业的产品略有差别，这样的市场结构被称为什么",
A 垄断竞争
B 完全竞争
C 寡头
D 垄断}"""

simple_prompt = f""" 
## 你的角色：Prompt工程师

## 任务：请你优化以下prompt 
{prompt}

## 输出格式：JSON
{{
    "优化之后的prompt"：<你优化之后的prompt>
}}
"""
prompt = """我有优化的prompt是：{我要采访一位科幻作家，创建一个包含 8 个问题的列表}
##不要生成我要的结果，不要回答prompt中的问题，只生成优化的prompt##"""
response = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": simple_prompt
                }],
            temperature = 0.00001)
r = response.body['result']

r = prompt_preprocess(r)

{
    "优化之后的prompt": "在经济学中，一个行业存在许多企业，这些企业销售的产品虽然相似但各有差异，这样的市场结构通常被称为什么？\n\nA. 垄断竞争\nB. 完全竞争\nC. 寡头\nD. 垄断"
}
